[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay04.ipynb)

# --- Day 4: Passport Processing ---

In [1]:
using System.IO;
using System.Text.RegularExpressions;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Read the input file and split into separate passport entries.

In [2]:
var batchFile = File.ReadAllText(@"input/04.txt").Split("\n\n");

Fields for the passport:

- byr (Birth Year)
- iyr (Issue Year)
- eyr (Expiration Year)
- hgt (Height)
- hcl (Hair Color)
- ecl (Eye Color)
- pid (Passport ID)
- cid (Country ID)

For Part 1, just check if the string contains each of the fields above, except for `cid`.

In [3]:
bool IsPassportValid(string passport)
{
    return passport.Contains("byr") &&
        passport.Contains("iyr") &&
        passport.Contains("eyr") &&
        passport.Contains("hgt") &&
        passport.Contains("hcl") &&
        passport.Contains("ecl") &&
        passport.Contains("pid");
}

In [4]:
batchFile.Where(IsPassportValid).Count()

264

# --- Part Two ---

For Part 2, create a proper `Passport` object. Here, decided to use C# 9's new `record` feature.

In [5]:
record Passport
{
    public int byr { get; init; }
    public int iyr { get; init; }
    public int eyr { get; init; }
    public string hgt { get; init; } = string.Empty;
    public string hcl { get; init; } = string.Empty;
    public string ecl { get; init; } = string.Empty;
    public string pid { get; init; } = string.Empty;
    public string cid { get; init; } = string.Empty;
    public bool IsValid
    {
        get
        {
            return IsByrValid(byr) &&
                IsIyrValid(iyr) &&
                IsEyrValid(eyr) &&
                IsHgtValid(hgt) &&
                IsHclValid(hcl) &&
                IsEclValid(ecl) &&
                IsPidValid(pid);
        }
    }
    
    public Passport(string passport)
    {
        Dictionary<string,string> tempDict = Regex
            .Split(passport, @"[ \r\n]")
            .Select(f => f.Split(":"))
            .ToDictionary(kv => kv.First(), kv => kv.Last());
        
        // Maybe possible to use reflection here?
        foreach (KeyValuePair<string, string> kv in tempDict)
        {
            switch (kv.Key)
            {
                case "byr":
                    byr = int.TryParse(kv.Value, out var ibyr) ? ibyr : 0;
                    break;
                case "iyr":
                    iyr = int.TryParse(kv.Value, out var iiyr) ? iiyr : 0;
                    break;
                case "eyr":
                    eyr = int.TryParse(kv.Value, out var ieyr) ? ieyr : 0;
                    break;
                case "hgt":
                    hgt = kv.Value;
                    break;
                case "hcl":
                    hcl = kv.Value;
                    break;
                case "ecl":
                    ecl = kv.Value;
                    break;
                case "pid":
                    pid = kv.Value;
                    break;
                case "cid":
                    cid = kv.Value;
                    break;
            }
        }
    }
    
    private bool IsByrValid(int byr) => 1920 <= byr && byr <= 2002;
    private bool IsIyrValid(int iyr) => 2010 <= iyr && iyr <= 2020;
    private bool IsEyrValid(int eyr) => 2020 <= eyr && eyr <= 2030;
    private bool IsHgtValid(string hgt)
    {
        if (hgt.EndsWith("cm"))
        {
            var cm = Convert.ToInt32(hgt.Substring(0, hgt.Length - 2));
            return 150 <= cm && cm <= 193;
        }
        if (hgt.EndsWith("in"))
        {
            var inches = Convert.ToInt32(hgt.Substring(0, hgt.Length - 2));
            return 59 <= inches && inches <= 76;
        }
        return false;
    }
    private bool IsHclValid(string hcl) => Regex.IsMatch(hcl, @"^#[0-9a-f]{6}$");
    private bool IsEclValid(string ecl) => Regex.IsMatch(ecl, @"^(amb|blu|brn|gry|grn|hzl|oth)$");
    private bool IsPidValid(string pid) => Regex.IsMatch(pid, @"^\d{9}$");
}

In [6]:
batchFile.Select(p => new Passport(p)).Where(p => p.IsValid).Count()

224